In [1]:
# import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import KMeans clutering class
from sklearn.cluster import KMeans

# Import silhouette_score function
from sklearn.metrics import silhouette_score 

## Load the data and apply preprocessing

### Load the data and merge the necessary information

In [ ]:
# load the datasets
sch_stat_df = pd.read_excel('Datasets/scottish_schools_stats.xlsx')
sch_info_df = pd.read_excel('Datasets/scottish_schools_contact.xlsx', sheet_name='Open Schools')
dep_df = pd.read_excel('Datasets/postcode_deprivation.xlsx')

In [ ]:
# change Seed Code name to be matched with the stat dataset
sch_info = sch_info_df.rename(columns={'Seed Code' : 'SeedCode'})

In [ ]:
# merge info and stat datasets
sch_df = sch_info.merge(sch_stat_df, on='SeedCode', how='left')

In [ ]:
columns_to_remove = ['School Name_y', 'Centre Type', 'Address Line1',
       'Address Line2', 'Address Line3', 'Unique Property Reference Number (UPRN)', 'Email', 'Phone Number',
       'Website Address', 'Pre-school Department', 'Primary Department',
       'Secondary Department', 'Special Department', 'Gaelic Unit',
       'Integrated Special Unit', 'Denomination', 'Local Authority']


sch_df.drop(columns=columns_to_remove, inplace=True)
sch_df = sch_df.rename(columns={'School Name_x' : 'School Name'})
sch_df = sch_df.rename(columns={'LA Name' : 'Local Authority'})

In [ ]:
# create a function to return the deprivation value
def get_dep(pos):
    try:
        dep_value = dep_df[dep_df['Postcode'] == pos]['SIMD2020v2_Quintile'].values[0]
        return dep_value
    
    except:
        return None

In [ ]:
# Add the deprivaion quintile value for each school
sch_df['Deprivation Quintile'] = list(map(get_dep, sch_df['Post Code']))

In [ ]:
# drop null values
sch_df = sch_df.dropna()

### Preprocessing

In [ ]:
# specify the numerical features
numeric_columns = ['Pupils', 'DeprivationScore']

# Import standard scaler to z-score normalize the data
from sklearn.preprocessing import StandardScaler

# Create a scaler object
scaler = StandardScaler()

# Scale the numeric columns
scaled_values = scaler.fit_transform(df[numeric_columns])

# Sonvert the numpy array of scaled values into a dataframe
scaled_values = pd.DataFrame(scaled_values, columns = numeric_columns)

# Describe the scaled columns: each has 0 mean and 1 standard deviation
scaled_values.describe().round(2)

## K-Means Clustering

### Find the best k - the number of clusters to be formed

Within-Cluster Sum of Squares (WCSS) method

In [ ]:
# Create a list to store WCSS values
wcss = []

# Iterate in a range from 2 to 10, inclusive
for k in range(2, 11):
    km = KMeans(n_clusters = k, n_init = 25, random_state = 1234) # Create a cluster object for each k
    km.fit(scaled_values) # Fit the scaled data
    wcss.append(km.inertia_) # Add the inertia score to wcss list

# Convert the wcss list into a pandas series object
wcss_series = pd.Series(wcss, index = range(2, 11))

# Draw a line chart showing the inertia score, or WCSS, for each k iterated
plt.figure(figsize=(8, 6))
ax = sns.lineplot(y = wcss_series, x = wcss_series.index)
ax = sns.scatterplot(y = wcss_series, x = wcss_series.index, s = 150)
ax = ax.set(xlabel = 'Number of Clusters (k)', 
            ylabel = 'Within Cluster Sum of Squares (WCSS)')

Average Silhouette Method

In [ ]:
# Create a list to store silhouette values
silhouette = []

# Iterate in a range from 2 to 10, inclusive
for k in range(2, 11):
    km = KMeans(n_clusters = k, n_init = 25, random_state = 1234) # Create a cluster object for each k
    km.fit(scaled_values) # Fit the scaled data
    silhouette.append(silhouette_score(scaled_values, km.labels_)) # Add the silhouette score to silhouette list

# Convert the silhouette list into a pandas series object
silhouette_series = pd.Series(silhouette, index = range(2, 11))

# Draw a line chart showing the average silhouette score for each k iterated
plt.figure(figsize=(8, 6))
ax = sns.lineplot(y = silhouette_series, x = silhouette_series.index, color='green')
ax = sns.scatterplot(y = silhouette_series, x = silhouette_series.index, s = 150, color='green')
ax = ax.set(xlabel = 'Number of Clusters (k)', 
            ylabel = 'Average Silhouette Score')